In this notebook you will find our results for section **2.2 Monte Carlo simulation of DLA**: 2 functions, 3 results in plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [ ]:
@njit
def stick(domain, n):
    '''
    This is my update version of Monte Carlo DLA model. if you looked at the previous ones you will know that tried using classes,
    then tracking the particle as a 1 value in my domain until it sticks. All those attempts where unsuccessful or very inefficient.
    Here I directly look for where the particule will stick to the existing cluster without bothering to track where the particle wonders.

    I create a top point and update it until it has a neighbor in the cluster (=2) then stop.
    '''

    #generate a particle
    angle = 2 * np.pi * np.random.rand()  
    y = int(n//2 + (n//3) * np.sin(angle))
    x = 0

    while True: 

        step = np.random.randint(4)  
        if step == 0:
            x_new, y_new = x - 1, y  # going up
        elif step == 1:
            x_new, y_new = x, y + 1  # going right
        elif step == 2:
            x_new, y_new = x + 1, y  # going down
        else:
            x_new, y_new = x, y - 1  # going left

        if x_new <= 0 or x_new >= n-1:
            return  # Particle leaves domain
        if y_new < 0:
            y_new = n-1
        if y_new > n-1:
            y_new = 0

        # if neighbor is part of the cluster then particle joins the cluster:
        if (domain[x_new+1, y_new] == 1 or domain[x_new, y_new+1] == 1 or
            domain[x_new-1, y_new] == 1 or domain[x_new, y_new-1] == 1):
            domain[x_new, y_new] = 1
            return
        
        x, y = x_new, y_new

In [ ]:
@njit
def stick_stoch(domain, n, p):
    '''
    Same function but only sticks with probability p
    '''
    #generate a particle
    angle = 2 * np.pi * np.random.random()  
    y = int(n//2 + (n//3) * np.sin(angle))
    x = 0

    while True: 
        
        dir = np.random.randint(4)  
        if dir == 0:
            x_new, y_new = x - 1, y  # going up
        elif dir == 1:
            x_new, y_new = x, y + 1  # going right
        elif dir == 2:
            x_new, y_new = x + 1, y  # going down
        else:
            x_new, y_new = x, y - 1  # going left

        if x_new <= 0 or x_new >= n-1:
            return  # Particle leaves domain
        if y_new < 0:
            y_new = n-1
        if y_new > n-1:
            y_new = 0

        if domain[x_new, y_new] == 1:
            continue
            #restart with new 'dir'

        # if neighbor is part of the cluster then particle joins the cluster:
        if (domain[(x_new + 1) % n, y_new] == 1 or
            domain[x_new, (y_new + 1) % n] == 1 or
            domain[(x_new - 1) % n, y_new] == 1 or
            domain[x_new, (y_new - 1) % n] == 1) and p > np.random.random():
            domain[x_new, y_new] = 1
            return
            
        x, y = x_new, y_new

In [ ]:
n = 100 # size of domain
cluster_size = 400 # size of cluster

The following kernel plots our Monte Carlo DLA model with deffinitive aggregation when a particle touches the cluster.

In [ ]:
domain = np.zeros((n, n), dtype=np.uint8)
domain[-1, n//2] = 1

while np.sum(domain != 0) < cluster_size:
    stick(domain, n)

im = plt.imshow(domain, interpolation='nearest')
plt.xlabel(f"x ({n})")
plt.ylabel(f"y ({n})")
plt.title(fr"Diffusion Limited Aggregation by Monte Carlo, cluster of {cluster_size} particles")
#plt.savefig("dla_set2_2B_MC.png", dpi = 300)
plt.show()

The following plots are runs of our MC DLA model for different probabilities for aggregation *p*, varying from $0.1$ to $0.9$. We use this to extrapolate a change in density of the cluster depending on this probability. 

In [ ]:
probs = np.linspace(0.1, 0.9, 9)  # probabilities to test
fig, axes = plt.subplots(3, 3, figsize=(15, 15))


for i, p in enumerate(probs):
    domain = np.zeros((n, n))
    domain[-1, n//2] = 1 

    while np.sum(domain != 0) < cluster_size:
        stick_stoch(domain, n, p)

    ax = axes.flat[i] 
    ax.imshow(domain, cmap="viridis") 
    ax.set_xlabel(f"x ({n})")
    ax.set_ylabel(f"y ({n})")
    ax.set_title(f"Probability to stick = {p:.1f}")

plt.tight_layout()
#plt.savefig("dla_set2_2B_MC_stoch_prob.png", dpi=300)
plt.show()